# 05. 軽量マルチエージェント基礎（Planner/Coder/Critic）

このノートブックでは、3役の役割分担で課題解決する最小パイプラインを体験します。


In [ ]:
# 編集禁止セル
import os
import sys

repo_path = '/content/llm_lab'
if os.path.exists(repo_path):
    !rm -rf {repo_path}
!git clone -b ai_agent https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.agent_tools import make_default_registry
from src.agent_core import RulePlannerCoderCriticAgent
print('セットアップ完了')


In [ ]:
# 編集禁止セル
registry = make_default_registry()
agent = RulePlannerCoderCriticAgent(registry)

queries = [
    '(23 + 17) * 5 を計算して',
    '今日の日付を教えて',
    '2026年2月の最新アニメ情報を要約して'
]

for q in queries:
    answer, steps = agent.run(q)
    print('=' * 70)
    print('Q:', q)
    print('A:', answer)
    for s in steps:
        print(f"- step{s.step_no} [{s.role}] action={s.action} observation={s.observation[:120]}")


## 演習

- `src/agent_core.py` の `_choose_action()` を改造し、ツール選択ルールを追加してください。
- `data/agent/sample_tasks.json` のタスクをすべて実行してください。
